In [1]:
import pandas as pd
import numpy as np
import time
import requests
from bs4 import BeautifulSoup
import re

In [2]:
westbrook = requests.get("https://www.basketball-reference.com/players/w/westbru01/gamelog/2016")

In [3]:
westbrook.status_code

200

In [4]:
westbrook.headers

{'Date': 'Fri, 01 Nov 2019 15:01:36 GMT', 'Content-Type': 'text/html; charset=UTF-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Keep-Alive': 'timeout=15', 'Vary': 'Accept-Encoding', 'SR-MISS-MEMD-KEY': '/players/w/westbru01/gamelog/2016', 'X-Accel-Expires': '28800', 'Source': 'back-1-bbr', 'X-cache-key-basic': 'www.basketball-reference.com/players/w/westbru01/gamelog/2016', 'X-Cache-Status': 'HIT', 'Content-Encoding': 'gzip'}

In [ ]:
westbrook.text

In [5]:
russ = BeautifulSoup(westbrook.text,'lxml')

In [ ]:
russ

In [6]:
games = russ.find("div", {"id": "all_pgl_basic"}).find_all('tr')

In [ ]:
games

In [7]:
headers_list = []

for i in games[0].find_all('th'):
    entry = i.getText()
    headers_list.append(entry)

In [8]:
headers_list

['Rk',
 'G',
 'Date',
 'Age',
 'Tm',
 '\xa0',
 'Opp',
 '\xa0',
 'GS',
 'MP',
 'FG',
 'FGA',
 'FG%',
 '3P',
 '3PA',
 '3P%',
 'FT',
 'FTA',
 'FT%',
 'ORB',
 'DRB',
 'TRB',
 'AST',
 'STL',
 'BLK',
 'TOV',
 'PF',
 'PTS',
 'GmSc',
 '+/-']

In [12]:
headers_list[7] = "WL"

In [14]:
headers_list[5] = "HA"

In [15]:
headers_list

['Rk',
 'G',
 'Date',
 'Age',
 'Tm',
 'HA',
 'Opp',
 'WL',
 'GS',
 'MP',
 'FG',
 'FGA',
 'FG%',
 '3P',
 '3PA',
 '3P%',
 'FT',
 'FTA',
 'FT%',
 'ORB',
 'DRB',
 'TRB',
 'AST',
 'STL',
 'BLK',
 'TOV',
 'PF',
 'PTS',
 'GmSc',
 '+/-']

In [70]:
stats = []

for i in games[74].find_all('td'):
    stats.append(i.getText())

In [71]:
stats

['71',
 '2016-03-22',
 '27-131',
 'OKC',
 '',
 'HOU',
 'W (+4)',
 '1',
 '36:52',
 '7',
 '18',
 '.389',
 '0',
 '7',
 '.000',
 '7',
 '8',
 '.875',
 '3',
 '9',
 '12',
 '15',
 '1',
 '0',
 '4',
 '1',
 '21',
 '22.7',
 '-5']

In [72]:
len(games)

87

In [41]:
def sea_stats(year):
    player_year = requests.get("https://www.basketball-reference.com/players/w/westbru01/gamelog/" + year)
    player_games = BeautifulSoup(player_year.text,'lxml')
    games = player_games.find("div", {"id": "all_pgl_basic"}).find_all('tr')
    
    headers = []
    season_stats = []

    for w in games[0].find_all('th'):
        entry = w.getText()
        headers.append(entry)

    headers[5] = "HA"
    headers[7] = "WL"
    
    headers = headers[1:] # To get headers
    
    for i in range(1,len(games)):
        game_stats = []
        for j in games[i].find_all('td'):
            game_stats.append(j.getText())
        season_stats.append(game_stats)
    
    season_stat = [x for x in season_stats if x] # To remove empty rows
    
    return pd.DataFrame(season_stat,columns=headers)

In [157]:
active_years = []
for i in range(2016,2021):
    active_years.append(str(i))

In [158]:
active_years

['2016', '2017', '2018', '2019', '2020']

In [46]:
list_of_seasons = []

for i in active_years:
    list_of_seasons.append(sea_stats(i))

In [48]:
result = pd.concat(list_of_seasons)

In [98]:
result.insert(1,"Name",)

In [99]:
result

G  Name        Date     Age   Tm HA  Opp       WL             GS     MP  \
0    1  Russ  2008-10-29  19-352  OKC     MIL  L (-11)              0  21:56   
1    2  Russ  2008-11-01  19-355  OKC  @  HOU  L (-12)              0  18:32   
2    3  Russ  2008-11-02  19-356  OKC     MIN   W (+3)              0  25:26   
3    4  Russ  2008-11-05  19-359  OKC     BOS  L (-13)              0  19:11   
4    5  Russ  2008-11-07  19-361  OKC  @  UTA   L (-7)              0  17:12   
5    6  Russ  2008-11-09  19-363  OKC     ATL   L (-4)              0  25:13   
6    7  Russ  2008-11-10  19-364  OKC  @  IND   L (-8)              0  25:35   
7    8  Russ  2008-11-12  20-000  OKC     ORL  L (-17)              0  30:02   
8    9  Russ  2008-11-14  20-002  OKC  @  NYK  L (-10)              0  27:37   
9   10  Russ  2008-11-15  20-003  OKC  @  PHI  L (-25)              0  23:48   
10  11  Russ  2008-11-17  20-005  OKC     HOU  L (-11)              0  28:03   
11  12  Russ  2008-11-19  20-007  OKC     LAC  L (-20)              0  30:03   
12  13  Russ  2008-11-21  20-009  OKC     NOH  L (-25)              0  28:46   
13  14  Russ  2008-11-22  20-010  OKC  @  NOH  L (-12)              0  30:18   
14  15  Russ  2008-11-25  20-013  OKC     PHO   L (-1)              0  30:52   
15  16  Russ  2008-11-26  20-014  OKC  @  CLE  L (-35)              0  29:04   
16  17  Russ  2008-11-28  20-016  OKC     MIN   L (-2)              0  34:27   
17  18  Russ  2008-11-29  20-017  OKC  @  MEM   W (+8)              1  38:04   
18  19  Russ  2008-12-03  20-021  OKC  @  CHA   L (-6)              1  36:06   
19  20  Russ  2008-12-05  20-023  OKC  @  ORL   L (-9)              1  33:36   
20  21  Russ  2008-12-06  20-024  OKC  @  MIA   L (-6)              1  32:02   
21  22  Russ  2008-12-08  20-026  OKC     GSW  L (-10)              1  34:05   
22  23  Russ  2008-12-10  20-028  OKC     MEM   L (-6)              1  32:44   
23  24  Russ  2008-12-13  20-031  OKC  @  DAL   L (-4)              1  34:56   
24  25  Russ  2008-12-14  20-032  OKC  @  SAS   L (-5)              1  19:06   
25  26  Russ  2008-12-16  20-034  OKC     LAC  L (-10)              1  32:40   
26  27  Russ  2008-12-19  20-037  OKC     TOR   W (+8)              1  31:26   
27  28  Russ  2008-12-21  20-039  OKC     CLE  L (-11)              1  40:03   
28  29  Russ  2008-12-23  20-041  OKC  @  ATL  L (-11)              1  37:52   
29  30  Russ  2008-12-26  20-044  OKC  @  DET   L (-2)              1  33:05   
..  ..   ...         ...     ...  ... ..  ...      ...            ...    ...   
56  49  Russ  2019-02-14  30-094  OKC  @  NOP   L (-9)              1  39:02   
57  50  Russ  2019-02-22  30-102  OKC     UTA   W (+1)              1  42:54   
58  51  Russ  2019-02-23  30-103  OKC     SAC   L (-3)              1  35:19   
59  52  Russ  2019-02-26  30-106  OKC  @  DEN   L (-9)              1  35:33   
60  53  Russ  2019-02-28  30-108  OKC     PHI   L (-4)              1  37:31   
61  54  Russ  2019-03-02  30-110  OKC  @  SAS  L (-14)              1  33:45   
62  55  Russ  2019-03-03  30-111  OKC     MEM   W (+4)              1  35:31   
63  56  Russ  2019-03-05  30-113  OKC  @  MIN  L (-11)              1  38:27   
64  57  Russ  2019-03-07  30-115  OKC  @  POR   W (+8)              1  40:04   
65  58  Russ  2019-03-08  30-116  OKC  @  LAC   L (-8)              1  31:58   
66  59  Russ  2019-03-11  30-119  OKC  @  UTA   W (+9)              1  37:20   
67  60  Russ  2019-03-13  30-121  OKC     BRK  W (+12)              1  35:49   
68  61  Russ  2019-03-14  30-122  OKC  @  IND   L (-2)              1  38:20   
69  62  Russ  2019-03-16  30-124  OKC     GSW  L (-22)              1  31:56   
70      Russ  2019-03-18  30-126  OKC     MIA   L (-9)  Not With Team   None   
71  63  Russ  2019-03-20  30-128  OKC     TOR   L (-9)              1  43:55   
72  64  Russ  2019-03-22  30-130  OKC  @  TOR   W (+7)              1  36:46   
73  65  Russ  2019-03-25  30-133  OKC  @  MEM  L (-12)              1  36:45   
74  6

In [42]:
sea_stats("2014")

G        Date     Age   Tm HA  Opp       WL            GS     MP    FG  \
0       2013-10-30  24-352  OKC  @  UTA   W (+3)      Inactive   None  None   
1       2013-11-01  24-354  OKC  @  MIN  L (-19)      Inactive   None  None   
2    1  2013-11-03  24-356  OKC     PHO   W (+7)             1  32:45     5   
3    2  2013-11-06  24-359  OKC     DAL  W (+14)             1  32:45    10   
4    3  2013-11-08  24-361  OKC  @  DET   W (+9)             1  30:05     5   
5    4  2013-11-10  24-363  OKC     WAS   W (+1)             1  30:01     4   
6    5  2013-11-13  25-001  OKC  @  LAC   L (-8)             1  34:49     7   
7    6  2013-11-14  25-002  OKC  @  GSW   L (-1)             1  35:58    13   
8    7  2013-11-16  25-004  OKC  @  MIL  W (+13)             1  33:14    10   
9    8  2013-11-18  25-006  OKC     DEN   W (+2)             1  38:06    10   
10   9  2013-11-21  25-009  OKC     LAC  W (+14)             1  31:21     5   
11      2013-11-24  25-012  OKC     UTA  W (+22)      Inactive   None  None   
12  10  2013-11-27  25-015  OKC     SAS   W (+6)             1  32:12     2   
13  11  2013-11-29  25-017  OKC     GSW   W (+1)             1  41:44    10   
14  12  2013-12-01  25-019  OKC     MIN  W (+10)             1  31:33     7   
15  13  2013-12-03  25-021  OKC  @  SAC   W (+2)             1  32:25     7   
16  14  2013-12-04  25-022  OKC  @  POR   L (-7)             1  29:33     7   
17  15  2013-12-06  25-024  OKC  @  NOP  W (+14)             1  33:37     8   
18  16  2013-12-08  25-026  OKC     IND  W (+24)             1  31:58    11   
19  17  2013-12-10  25-028  OKC  @  ATL   W (+9)             1  35:15     6   
20  18  2013-12-11  25-029  OKC  @  MEM  W (+16)             1  27:26     7   
21  19  2013-12-13  25-031  OKC     LAL  W (+25)             1  30:54     7   
22  20  2013-12-15  25-033  OKC     ORL   W (+3)             1  33:09     7   
23  21  2013-12-17  25-035  OKC  @  DEN  W (+12)             1  36:13     9   
24  22  2013-12-19  25-037  OKC     CHI  W (+12)             1  27:52     7   
25  23  2013-12-21  25-039  OKC  @  SAS  W (+13)             1  34:17    13   
26  24  2013-12-22  25-040  OKC     TOR   L (-6)             1  35:13     8   
27  25  2013-12-25  25-043  OKC  @  NYK  W (+29)             1  29:09     5   
28      2013-12-27  25-045  OKC  @  CHA   W (+4)      Inactive   None  None   
29      2013-12-29  25-047  OKC     HOU  W (+31)      Inactive   None  None   
..  ..         ...     ...  ... ..  ...      ...           ...    ...   ...   
52      2014-02-09  25-089  OKC     NYK  W (+12)      Inactive   None  None   
53      2014-02-11  25-091  OKC  @  POR   W (+3)      Inactive   None  None   
54      2014-02-13  25-093  OKC  @  LAL   W (+4)      Inactive   None  None   
55  26  2014-02-20  25-100  OKC     MIA  L (-22)             1  23:53     3   
56  27  2014-02-23  25-103  OKC     LAC   L (-8)             1  24:49     3   
57  28  2014-02-26  25-106  OKC     CLE  L (-10)             1  29:59     8   
58  29  2014-02-28  25-108  OKC     MEM   W (+6)             1  29:27     7   
59  30  2014-03-02  25-110  OKC     CHA  W (+17)             1  25:15    10   
60  31  2014-03-04  25-112  OKC     PHI  W (+33)             1  20:17     4   
61  32  2014-03-06  25-114  OKC  @  PHO   L (-6)             1  30:28    13   
62  33  2014-03-09  25-117  OKC  @  LAL   L (-4)             1  30:32     7   
63  34  2014-03-11  25-119  OKC     HOU   W (+8)             1  30:08     6   
64  35  2014-03-13  25-121  OKC     LAL  W (+29)             1  23:20     9   
65      2014-03-16  25-124  OKC     DAL  L (-23)  Did Not Play   None  None   
66  36  2014-03-17  25-125  OKC  @  CHI  W (+12)             1  28:16     5   
67      2014-03-20  25-128  OKC  @  CLE   W (+7)  Did Not Play   None  None   
68  37  2014-03-21  25-129  OKC  @  TOR   W (+1)             1  19:11     5   
69      2014-03-24  25-132  OKC     DEN  W (+21)  Did Not Play   None  None   
70  38  2014-03-25  25-133  OKC  @  DAL   L (-9)       

In [37]:
sea_stats("2014")

None


In [25]:
headers = headers[1:]

In [26]:
headers

['G',
 'Date',
 'Age',
 'Tm',
 'HA',
 'Opp',
 'WL',
 'GS',
 'MP',
 'FG',
 'FGA',
 'FG%',
 '3P',
 '3PA',
 '3P%',
 'FT',
 'FTA',
 'FT%',
 'ORB',
 'DRB',
 'TRB',
 'AST',
 'STL',
 'BLK',
 'TOV',
 'PF',
 'PTS',
 'GmSc',
 '+/-']

In [29]:
season_stat

[['1',
  '2015-10-28',
  '26-350',
  'OKC',
  '',
  'SAS',
  'W (+6)',
  '1',
  '36:59',
  '12',
  '23',
  '.522',
  '3',
  '6',
  '.500',
  '6',
  '7',
  '.857',
  '1',
  '1',
  '2',
  '10',
  '2',
  '0',
  '5',
  '3',
  '33',
  '25.1',
  '+7'],
 ['2',
  '2015-10-30',
  '26-352',
  'OKC',
  '@',
  'ORL',
  'W (+3)',
  '1',
  '47:35',
  '17',
  '36',
  '.472',
  '1',
  '5',
  '.200',
  '13',
  '16',
  '.813',
  '5',
  '6',
  '11',
  '8',
  '1',
  '1',
  '6',
  '3',
  '48',
  '33.8',
  '+2'],
 ['3',
  '2015-11-01',
  '26-354',
  'OKC',
  '',
  'DEN',
  'W (+24)',
  '1',
  '25:26',
  '7',
  '13',
  '.538',
  '1',
  '4',
  '.250',
  '0',
  '0',
  '',
  '2',
  '7',
  '9',
  '8',
  '3',
  '0',
  '4',
  '0',
  '15',
  '16.8',
  '+22'],
 ['4',
  '2015-11-02',
  '26-355',
  'OKC',
  '@',
  'HOU',
  'L (-5)',
  '1',
  '31:57',
  '10',
  '16',
  '.625',
  '3',
  '6',
  '.500',
  '2',
  '2',
  '1.000',
  '2',
  '6',
  '8',
  '11',
  '2',
  '0',
  '7',
  '5',
  '25',
  '21.7',
  '+4'],
 ['5',
  '2

In [30]:
pd.DataFrame(season_stat,columns=headers)

G        Date     Age   Tm HA  Opp       WL             GS     MP    FG  \
0    1  2015-10-28  26-350  OKC     SAS   W (+6)              1  36:59    12   
1    2  2015-10-30  26-352  OKC  @  ORL   W (+3)              1  47:35    17   
2    3  2015-11-01  26-354  OKC     DEN  W (+24)              1  25:26     7   
3    4  2015-11-02  26-355  OKC  @  HOU   L (-5)              1  31:57    10   
4    5  2015-11-04  26-357  OKC     TOR   L (-5)              1  35:20     8   
5    6  2015-11-05  26-358  OKC  @  CHI   L (-6)              1  38:34     7   
6    7  2015-11-08  26-361  OKC     PHO  W (+21)              1  33:58     9   
7    8  2015-11-10  26-363  OKC  @  WAS  W (+24)              1  27:46     7   
8    9  2015-11-13  27-001  OKC     PHI  W (+17)              1  34:47     8   
9   10  2015-11-15  27-003  OKC     BOS  L (-15)              1  32:37     5   
10  11  2015-11-16  27-004  OKC  @  MEM   L (-8)              1  38:32    13   
11  12  2015-11-18  27-006  OKC     NOP   W (+7)              1  36:04    14   
12  13  2015-11-20  27-008  OKC     NYK   L (-3)              1  37:37    11   
13  14  2015-11-22  27-010  OKC     DAL   W (+3)              1  36:53    12   
14  15  2015-11-23  27-011  OKC  @  UTA  W (+22)              1  27:18     5   
15  16  2015-11-25  27-013  OKC     BRK  W (+11)              1  35:14    10   
16  17  2015-11-27  27-015  OKC     DET  W (+16)              1  28:37     5   
17  18  2015-11-30  27-018  OKC  @  ATL   L (-6)              1  35:57    11   
18  19  2015-12-03  27-021  OKC  @  MIA   L (-2)              1  34:27     8   
19  20  2015-12-06  27-024  OKC     SAC   W (+3)              1  37:52     7   
20  21  2015-12-08  27-026  OKC  @  MEM  W (+37)              1  24:56     5   
21  22  2015-12-10  27-028  OKC     ATL  W (+13)              1  38:28     8   
22  23  2015-12-11  27-029  OKC  @  UTA   W (+4)              1  35:02     8   
23  24  2015-12-13  27-031  OKC     UTA   W (+6)              1  40:22     9   
24  25  2015-12-16  27-034  OKC     POR  W (+16)              1  26:53     5   
25  26  2015-12-17  27-035  OKC  @  CLE   L (-4)              1  34:28    10   
26  27  2015-12-19  27-037  OKC     LAL  W (+40)              1  27:43     5   
27  28  2015-12-21  27-039  OKC  @  LAC   W (+1)              1  37:19    11   
28  29  2015-12-23  27-041  OKC  @  LAL  W (+35)              1  29:33    10   
29  30  2015-12-25  27-043  OKC     CHI   L (-9)              1  39:57     9   
..  ..         ...     ...  ... ..  ...      ...            ...    ...   ...   
52  53  2016-02-08  27-088  OKC  @  PHO  W (+16)              1  36:14     8   
53  54  2016-02-11  27-091  OKC     NOP  W (+26)              1  28:25     8   
54  55  2016-02-19  27-099  OKC     IND   L (-3)              1  37:09    10   
55  56  2016-02-21  27-101  OKC     CLE  L (-23)              1  33:04     7   
56  57  2016-02-24  27-104  OKC  @  DAL  W (+13)              1  36:09     8   
57  58  2016-02-25  27-105  OKC  @  NOP   L (-4)              1  37:37    15   
58  59  2016-02-27  27-107  OKC     GSW   L (-3)              1  42:11    10   
59  60  2016-02-29  27-109  OKC  @  SAC  W (+15)              1  34:35     6   
60  61  2016-03-02  27-111  OKC  @  LAC   L (-5)              1  36:51     8   
61  62  2016-03-03  27-112  OKC  @  GSW  L (-15)              1  36:03     8   
62  63  2016-03-06  27-115  OKC  @  MIL   W (+8)              1  38:50     3   
63  64  2016-03-09  27-118  OKC     LAC  W (+12)              1  37:02     9   
64  65  2016-03-11  27-120  OKC     MIN   L (-3)              1  37:10     8   
65  66  2016-03-12  27-121  OKC  @  SAS   L (-8)              1  38:14     5   
66  67  2016-03-14  27-123  OKC     POR  W (+34)              1  28:27     7   
67  68  2016-03-16  27-125  OKC  @  BOS  W (+21)              1  26:09     8   
68  69  2016-03-18  27-127  OKC  @  PHI  W (+14)              1  32:26     8   
69  70  2016-03-19  27-128  OKC  @  IND   W (+4)              1  35:56     4   
70  7

# Getting all players who are playing in current 2020 season

In [100]:
player_current_page = requests.get("https://www.basketball-reference.com/leagues/NBA_2020_per_game.html")
player_names = BeautifulSoup(player_current_page.text,'lxml')
players = player_names.find("div", {"id": "all_per_game_stats"}).find_all('tr')

In [101]:
links_names = []
for i in range(len(players)):
    try:
        for a in players[i].find_all('td')[0].find_all('a',href=True):
            links_names.append(a['href'].split('.')[0])
    except:
        IndexError

In [ ]:
links_names

In [104]:
links_names_test = links_names[:10]

['/players/a/adamsst01',
 '/players/a/adebaba01',
 '/players/a/aldrila01',
 '/players/a/alexani01',
 '/players/a/allengr01',
 '/players/a/allenja01',
 '/players/a/aminual01',
 '/players/a/anderky01',
 '/players/a/anderry01',
 '/players/a/antetgi01']

In [159]:
def sea_stats(player,year):
    playa = requests.get("https://www.basketball-reference.com"+player+"/gamelog/"+year)
    #player_year = requests.get("https://www.basketball-reference.com/players/w/westbru01/gamelog/" + year)
    player_games = BeautifulSoup(playa.text,'lxml')
    games = player_games.find("div", {"id": "all_pgl_basic"}).find_all('tr')
    
    headers = []
    season_stats = []

    for w in games[0].find_all('th'):
        entry = w.getText()
        headers.append(entry)

    headers[5] = "HA"
    headers[7] = "WL"
    
    headers = headers[1:] # To get headers
    
    for i in range(1,len(games)):
        game_stats = []
        for j in games[i].find_all('td'):
            game_stats.append(j.getText())
        season_stats.append(game_stats)
    
    season_stat = [x for x in season_stats if x] # To remove empty rows
    
    return pd.DataFrame(season_stat,columns=headers)

In [160]:
def all_seasons(player):
    
    list_of_seasons = []
    
    for i in active_years:
        try:
            stats = sea_stats(player,i)
            list_of_seasons.append(stats)
        except:
            AttributeError
            
    result = pd.concat(list_of_seasons)
    
    player_name = player.split("/")[3]
    
    result.insert(0,"Name",player_name)
    
    return result

In [162]:
all_players = []
for name in links_names_test:
    all_players.append(all_seasons(name))

In [163]:
pd.concat(all_players)

Name   G        Date     Age   Tm HA  Opp       WL             GS  \
0   adamsst01   1  2015-10-28  22-100  OKC     SAS   W (+6)              1   
1   adamsst01   2  2015-10-30  22-102  OKC  @  ORL   W (+3)              1   
2   adamsst01   3  2015-11-01  22-104  OKC     DEN  W (+24)              1   
3   adamsst01   4  2015-11-02  22-105  OKC  @  HOU   L (-5)              1   
4   adamsst01   5  2015-11-04  22-107  OKC     TOR   L (-5)              1   
5   adamsst01   6  2015-11-05  22-108  OKC  @  CHI   L (-6)              1   
6   adamsst01   7  2015-11-08  22-111  OKC     PHO  W (+21)              1   
7   adamsst01   8  2015-11-10  22-113  OKC  @  WAS  W (+24)              1   
8   adamsst01   9  2015-11-13  22-116  OKC     PHI  W (+17)              1   
9   adamsst01  10  2015-11-15  22-118  OKC     BOS  L (-15)              1   
10  adamsst01  11  2015-11-16  22-119  OKC  @  MEM   L (-8)              1   
11  adamsst01  12  2015-11-18  22-121  OKC     NOP   W (+7)              1   
12  adamsst01  13  2015-11-20  22-123  OKC     NYK   L (-3)              1   
13  adamsst01  14  2015-11-22  22-125  OKC     DAL   W (+3)              1   
14  adamsst01  15  2015-11-23  22-126  OKC  @  UTA  W (+22)              1   
15  adamsst01  16  2015-11-25  22-128  OKC     BRK  W (+11)              1   
16  adamsst01  17  2015-11-27  22-130  OKC     DET  W (+16)              1   
17  adamsst01  18  2015-11-30  22-133  OKC  @  ATL   L (-6)              1   
18  adamsst01  19  2015-12-03  22-136  OKC  @  MIA   L (-2)              1   
19  adamsst01  20  2015-12-06  22-139  OKC     SAC   W (+3)              1   
20  adamsst01  21  2015-12-08  22-141  OKC  @  MEM  W (+37)              1   
21  adamsst01  22  2015-12-10  22-143  OKC     ATL  W (+13)              1   
22  adamsst01  23  2015-12-11  22-144  OKC  @  UTA   W (+4)              1   
23  adamsst01  24  2015-12-13  22-146  OKC     UTA   W (+6)              1   
24  adamsst01  25  2015-12-16  22-149  OKC     POR  W (+16)              1   
25  adamsst01  26  2015-12-17  22-150  OKC  @  CLE   L (-4)              1   
26  adamsst01  27  2015-12-19  22-152  OKC     LAL  W (+40)              1   
27  adamsst01  28  2015-12-21  22-154  OKC  @  LAC   W (+1)              1   
28  adamsst01  29  2015-12-23  22-156  OKC  @  LAL  W (+35)              1   
29  adamsst01  30  2015-12-25  22-158  OKC     CHI   L (-9)              1   
..        ...  ..         ...     ...  ... ..  ...      ...            ...   
56  antetgi01  53  2019-02-13  24-069  MIL  @  IND   W (+9)              1   
57  antetgi01  54  2019-02-21  24-077  MIL     BOS   W (+1)              1   
58  antetgi01  55  2019-02-23  24-079  MIL     MIN  W (+12)              1   
59  antetgi01      2019-02-25  24-081  MIL  @  CHI  W (+11)       Inactive   
60  antetgi01  56  2019-02-27  24-083  MIL  @  SAC   W (+1)              1   
61  antetgi01  57  2019-03-01  24-085  MIL  @  LAL  W (+11)              1   
62  antetgi01  58  2019-03-02  24-086  MIL  @  UTA   L (-4)              1   
63  antetgi01  59  2019-03-04  24-088  MIL  @  PHO   L (-9)              1   
64  antetgi01  60  2019-03-07  24-091  MIL     IND  W (+19)              1   
65  antetgi01  61  2019-03-09  24-093  MIL     CHO  W (+17)              1   
66  antetgi01  62  2019-03-10  24-094  MIL  @  SAS   L (-7)              1   
67  antetgi01  63  2019-03-12  24-096  MIL  @  NOP  W (+17)              1   
68  antetgi01  64  2019-03-15  24-099  MIL  @  MIA  W (+15)              1   
69  antetgi01  65  2019-03-17  24-101  MIL     PHI   L (-5)              1   
70  antetgi01      2019-03-19  24-103  MIL     LAL  W (+14)   Did Not Play   
71  antetgi01      2019-03-20  24-104  MIL  @  CLE   L (-5)   Did Not Play   
72  antetgi01  66  2019-03-22  24-106  MIL     MIA  W (+29)              1   
73  antetgi01  67  2019-03-24  24-108  MIL     CLE  W (+22)              1   
74  antetgi01  68  2019-03-26  24-110  MIL     HOU  W (+14)              1   
75  antetgi01  69  2019-03-28  

In [161]:
all_seasons(links_names_test[0])

Name   G        Date     Age   Tm HA  Opp       WL        GS     MP  \
0   adamsst01   1  2015-10-28  22-100  OKC     SAS   W (+6)         1  29:34   
1   adamsst01   2  2015-10-30  22-102  OKC  @  ORL   W (+3)         1  35:54   
2   adamsst01   3  2015-11-01  22-104  OKC     DEN  W (+24)         1  21:32   
3   adamsst01   4  2015-11-02  22-105  OKC  @  HOU   L (-5)         1  19:12   
4   adamsst01   5  2015-11-04  22-107  OKC     TOR   L (-5)         1  26:36   
5   adamsst01   6  2015-11-05  22-108  OKC  @  CHI   L (-6)         1  24:58   
6   adamsst01   7  2015-11-08  22-111  OKC     PHO  W (+21)         1  19:31   
7   adamsst01   8  2015-11-10  22-113  OKC  @  WAS  W (+24)         1  20:36   
8   adamsst01   9  2015-11-13  22-116  OKC     PHI  W (+17)         1  25:59   
9   adamsst01  10  2015-11-15  22-118  OKC     BOS  L (-15)         1  29:35   
10  adamsst01  11  2015-11-16  22-119  OKC  @  MEM   L (-8)         1  26:49   
11  adamsst01  12  2015-11-18  22-121  OKC     NOP   W (+7)         1  11:24   
12  adamsst01  13  2015-11-20  22-123  OKC     NYK   L (-3)         1  21:17   
13  adamsst01  14  2015-11-22  22-125  OKC     DAL   W (+3)         1  24:33   
14  adamsst01  15  2015-11-23  22-126  OKC  @  UTA  W (+22)         1  23:14   
15  adamsst01  16  2015-11-25  22-128  OKC     BRK  W (+11)         1  21:44   
16  adamsst01  17  2015-11-27  22-130  OKC     DET  W (+16)         1  27:55   
17  adamsst01  18  2015-11-30  22-133  OKC  @  ATL   L (-6)         1  14:05   
18  adamsst01  19  2015-12-03  22-136  OKC  @  MIA   L (-2)         1  20:13   
19  adamsst01  20  2015-12-06  22-139  OKC     SAC   W (+3)         1  28:31   
20  adamsst01  21  2015-12-08  22-141  OKC  @  MEM  W (+37)         1  14:25   
21  adamsst01  22  2015-12-10  22-143  OKC     ATL  W (+13)         1  30:47   
22  adamsst01  23  2015-12-11  22-144  OKC  @  UTA   W (+4)         1  24:07   
23  adamsst01  24  2015-12-13  22-146  OKC     UTA   W (+6)         1  32:26   
24  adamsst01  25  2015-12-16  22-149  OKC     POR  W (+16)         1  21:41   
25  adamsst01  26  2015-12-17  22-150  OKC  @  CLE   L (-4)         1  27:16   
26  adamsst01  27  2015-12-19  22-152  OKC     LAL  W (+40)         1  24:35   
27  adamsst01  28  2015-12-21  22-154  OKC  @  LAC   W (+1)         1  25:45   
28  adamsst01  29  2015-12-23  22-156  OKC  @  LAL  W (+35)         1  24:36   
29  adamsst01  30  2015-12-25  22-158  OKC     CHI   L (-9)         1  12:45   
..        ...  ..         ...     ...  ... ..  ...      ...       ...    ...   
57  adamsst01  56  2019-02-22  25-217  OKC     UTA   W (+1)         1  47:08   
58  adamsst01  57  2019-02-23  25-218  OKC     SAC   L (-3)         1  27:54   
59  adamsst01  58  2019-02-26  25-221  OKC  @  DEN   L (-9)         1  30:29   
60  adamsst01  59  2019-02-28  25-223  OKC     PHI   L (-4)         1  31:39   
61  adamsst01  60  2019-03-02  25-225  OKC  @  SAS  L (-14)         1  24:57   
62  adamsst01  61  2019-03-03  25-226  OKC     MEM   W (+4)         1  32:21   
63  adamsst01  62  2019-03-05  25-228  OKC  @  MIN  L (-11)         1  36:29   
64  adamsst01  63  2019-03-07  25-230  OKC  @  POR   W (+8)         1  38:47   
65  adamsst01  64  2019-03-08  25-231  OKC  @  LAC   L (-8)         1  31:42   
66  adamsst01  65  2019-03-11  25-234  OKC  @  UTA   W (+9)         1  33:28   
67  adamsst01  66  2019-03-13  25-236  OKC     BRK  W (+12)         1  30:28   
68  adamsst01  67  2019-03-14  25-237  OKC  @  IND   L (-2)         1  33:52   
69  adamsst01  68  2019-03-16  25-239  OKC     GSW  L (-22)         1  33:16   
70  adamsst01  69  2019-03-18  25-241  OKC     MIA   L (-9)         1  24:38   
71  adamsst01  70  2019-03-20  25-243  OKC     TOR   L (-9)         1  39:58   
72  adamsst01  71  2019-03-22  25-245  OKC  @  TOR   W (+7)         1  33:08   
73  adamsst01  72  2019-03-25  25-248  OKC  @  MEM  L (-12)         1  31:54   
74  adamsst01  73  2019-03-27  25-250  OKC     IND   W (+8)         1  34:46   
75  adams

In [ ]:
active_years = []
for i in range(1996,2021):
    active_years.append(str(i))

In [ ]:
list_of_seasons = []

for i in active_years:
    list_of_seasons.append(sea_stats(i))

In [ ]:
result = pd.concat(list_of_seasons)

In [60]:
my_list = []
for i in games[1].find_all('td'):
    my_list.append(i)

In [111]:
player_year = requests.get("https://www.basketball-reference.com/players/w/westbru01/gamelog/" + "1999")
player_games = BeautifulSoup(player_year.text,'lxml')
try:
    games = player_games.find("div", {"id": "all_pgl_basic"}).find_all('tr')
except:
    AttributeError

In [109]:
player_year

<Response [200]>

In [61]:
my_list

['Steven Adams',
 'C',
 '26',
 'OKC',
 '4',
 '4',
 '27.3',
 '2.5',
 '8.0',
 '.313',
 '0.0',
 '0.0',
 '',
 '2.5',
 '8.0',
 '.313',
 '.313',
 '1.0',
 '3.5',
 '.286',
 '3.5',
 '8.3',
 '11.8',
 '1.5',
 '0.5',
 '0.8',
 '1.3',
 '2.5',
 '6.0']

In [68]:
games.find_all('a', href=True)

AttributeError: ResultSet object has no attribute 'find_all'. You're probably treating a list of items like a single item. Did you call find_all() when you meant to call find()?